In [50]:
from nltk.corpus import reuters 
 
def collection_stats():
    # List of documents
    documents = reuters.fileids()
    print(str(len(documents)) + " documents")
 
    train_docs = list(filter(lambda doc: doc.startswith("train"),
                        documents))
    
    test_docs = list(filter(lambda doc: doc.startswith("test"),
                       documents))
    
    # List of categories
    categories = reuters.categories()
    print(str(len(categories)) + " categories")
    
    print categories
 
    # Documents in a category
    #category_docs = reuters.fileids("money-supply")
 
    # Words for a document
    # document_id = category_docs[2]
    # document_words = reuters.words(category_docs[2])
    # print(document_words)  
 
    # Raw document
    # print(reuters.raw(document_id))

In [51]:
collection_stats()

10788 documents
90 categories
[u'acq', u'alum', u'barley', u'bop', u'carcass', u'castor-oil', u'cocoa', u'coconut', u'coconut-oil', u'coffee', u'copper', u'copra-cake', u'corn', u'cotton', u'cotton-oil', u'cpi', u'cpu', u'crude', u'dfl', u'dlr', u'dmk', u'earn', u'fuel', u'gas', u'gnp', u'gold', u'grain', u'groundnut', u'groundnut-oil', u'heat', u'hog', u'housing', u'income', u'instal-debt', u'interest', u'ipi', u'iron-steel', u'jet', u'jobs', u'l-cattle', u'lead', u'lei', u'lin-oil', u'livestock', u'lumber', u'meal-feed', u'money-fx', u'money-supply', u'naphtha', u'nat-gas', u'nickel', u'nkr', u'nzdlr', u'oat', u'oilseed', u'orange', u'palladium', u'palm-oil', u'palmkernel', u'pet-chem', u'platinum', u'potato', u'propane', u'rand', u'rape-oil', u'rapeseed', u'reserves', u'retail', u'rice', u'rubber', u'rye', u'ship', u'silver', u'sorghum', u'soy-meal', u'soy-oil', u'soybean', u'strategic-metal', u'sugar', u'sun-meal', u'sun-oil', u'sunseed', u'tea', u'tin', u'trade', u'veg-oil', u'whe

In [52]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
import re
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [4]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer


# Return the representer, without transforming
def tf_idf(docs):
    tfidf = TfidfVectorizer(tokenizer=tokenize_and_stem, min_df=0.01,
                        max_df=0.90, max_features=3000,
                        use_idf=True, sublinear_tf=True,
                        norm='l2');
    tfidf.fit(docs);
    return tfidf;

# max_df=0.9, max_features=200000,
# min_df=0.01, stop_words='english',
# use_idf=True, tokenizer=tokenize_and_stem, 
# ngram_range=(1,3)



In [53]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=tokenize_and_stem, min_df=0.01,
                        max_df=0.90, max_features=3000,
                        use_idf=True, sublinear_tf=True,
                        norm='l2');

In [5]:
def feature_values(doc, representer):
    doc_representation = representer.transform([doc])
    features = representer.get_feature_names()
    return [(features[index], doc_representation[0, index])
                 for index in doc_representation.nonzero()[1]]

In [54]:
train_docs = []
test_docs = []
for doc_id in reuters.fileids():
    if doc_id.startswith("train"):
        train_docs.append(reuters.raw(doc_id))
    else:
        test_docs.append(reuters.raw(doc_id))

In [55]:
print reuters.fileids()[0]

test/14826


In [8]:
#representer = tf_idf(train_docs);

In [ ]:
#for doc in test_docs:
#    print(feature_values(doc, representer))

In [56]:
vectorised_train_documents = tfidf.fit_transform(train_docs)

In [57]:
from sklearn.metrics.pairwise import cosine_similarity
X = 1 - cosine_similarity(vectorised_train_documents)

In [58]:
from sklearn.cluster import AffinityPropagation
from sklearn import metrics

# Compute Affinity Propagation
af = AffinityPropagation(damping=0.6).fit(X)
cluster_centers_indices = af.cluster_centers_indices_
labels = af.labels_

n_clusters_ = len(cluster_centers_indices)

print('Estimated number of clusters: %d' % n_clusters_)
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels, metric='cosine'))
print("Calinski_Harabaz Coefficient: %0.3f"
      % metrics.calinski_harabaz_score(X, labels))

Estimated number of clusters: 238
Silhouette Coefficient: 0.109
Calinski_Harabaz Coefficient: 261.117


In [ ]:
# it finds much more clusters than what we have (238 vs 90)
# Silhouette Coefficient is very low indicating that clusters overlap a lot

In [ ]:
# selecting only 244 documents at random

In [68]:
rndm_reuters_fileids_train = []
randint_train = []
#print random.uniform(0, len(rndm_reuters_fileids))
import numpy as np
for i in range(0,250):
    randint = np.random.randint(0,len(reuters.fileids()))
    randint_train.append(randint)
    rndm_reuters_fileids_train.append(reuters.fileids()[randint])
print rndm_reuters_fileids_train

['training/10708', 'training/6452', 'training/2047', 'test/20533', 'test/20480', 'test/19860', 'training/3997', 'test/19947', 'training/9373', 'training/6399', 'test/16041', 'training/5738', 'training/5640', 'training/9815', 'test/20593', 'training/12433', 'training/4067', 'test/16076', 'training/10760', 'training/10498', 'training/9734', 'test/18313', 'training/3593', 'test/16428', 'training/7659', 'training/11335', 'training/81', 'training/13382', 'training/6239', 'training/5416', 'training/374', 'test/18973', 'training/2159', 'training/5410', 'training/2029', 'training/330', 'training/1708', 'training/8986', 'test/19675', 'test/20052', 'training/2802', 'test/21157', 'test/16041', 'training/745', 'training/5002', 'training/5683', 'training/4049', 'training/14402', 'training/6588', 'training/2622', 'test/16646', 'training/2425', 'test/19005', 'training/12848', 'training/1188', 'training/1949', 'test/20045', 'test/18599', 'training/5543', 'training/1094', 'training/4474', 'training/269

In [60]:
rndm_reuters_fileids_test = []
randint_test = []
#print random.uniform(0, len(rndm_reuters_fileids))
import numpy as np
for i in range(0,80):
    randint = np.random.randint(0,len(reuters.fileids()))
    randint_test.append(randint)
    rndm_reuters_fileids_test.append(reuters.fileids()[randint])
print rndm_reuters_fileids_test

['test/21401', 'test/20981', 'test/20769', 'training/9497', 'training/13539', 'training/7266', 'test/16675', 'test/21260', 'test/18604', 'test/15866', 'training/8605', 'test/15778', 'training/2919', 'training/12344', 'training/3695', 'training/12222', 'training/5458', 'training/2896', 'training/8200', 'test/20652', 'training/1528', 'training/7558', 'test/18570', 'training/9356', 'training/10106', 'test/20730', 'test/16493', 'test/19946', 'test/21248', 'test/14985', 'training/7113', 'training/14099', 'training/6093', 'test/16175', 'training/9511', 'test/18851', 'test/15483', 'training/5690', 'training/5477', 'test/15808', 'training/3713', 'training/10973', 'training/8027', 'training/1047', 'training/12258', 'training/9893', 'test/17643', 'training/4152', 'training/4831', 'test/15914', 'test/14912', 'training/8478', 'training/7634', 'test/16357', 'training/4759', 'training/7873', 'training/11751', 'training/14315', 'training/6442', 'training/3620', 'training/10404', 'training/5411', 'tra

In [69]:
train_docs_160 = []
test_docs_80 = []
for doc_id in rndm_reuters_fileids_train:
    train_docs_160.append(reuters.raw(doc_id))
for doc_id in rndm_reuters_fileids_test:
    test_docs_80.append(reuters.raw(doc_id))

In [62]:
vectorised_test_documents_80 = tfidf.transform(test_docs_80)

In [70]:
vectorised_train_documents_160 = tfidf.transform(train_docs_160)

In [71]:
from sklearn.metrics.pairwise import cosine_similarity
X = 1 - cosine_similarity(vectorised_train_documents_160)

In [72]:
from sklearn.cluster import AffinityPropagation
from sklearn import metrics

# Compute Affinity Propagation
for damp in np.arange(0.5,1.0,0.05):
    af = AffinityPropagation(damping=damp).fit(X)
    cluster_centers_indices = af.cluster_centers_indices_
    labels = af.labels_

    n_clusters_ = len(cluster_centers_indices)
    print('Damping: ',damp)
    print('Estimated number of clusters: %d' % n_clusters_)
    print("Silhouette Coefficient: %0.3f"
        % metrics.silhouette_score(X, labels, metric='cosine'))
    print("Calinski_Harabaz Coefficient: %0.3f"
        % metrics.calinski_harabaz_score(X, labels))

('Damping: ', 0.5)
Estimated number of clusters: 30
Silhouette Coefficient: 0.180
Calinski_Harabaz Coefficient: 22.344
('Damping: ', 0.55000000000000004)
Estimated number of clusters: 32
Silhouette Coefficient: 0.180
Calinski_Harabaz Coefficient: 21.527
('Damping: ', 0.60000000000000009)
Estimated number of clusters: 31
Silhouette Coefficient: 0.179
Calinski_Harabaz Coefficient: 21.882
('Damping: ', 0.65000000000000013)
Estimated number of clusters: 31
Silhouette Coefficient: 0.179
Calinski_Harabaz Coefficient: 21.965
('Damping: ', 0.70000000000000018)
Estimated number of clusters: 31
Silhouette Coefficient: 0.179
Calinski_Harabaz Coefficient: 21.965
('Damping: ', 0.75000000000000022)
Estimated number of clusters: 31
Silhouette Coefficient: 0.179
Calinski_Harabaz Coefficient: 21.965
('Damping: ', 0.80000000000000027)
Estimated number of clusters: 31
Silhouette Coefficient: 0.179
Calinski_Harabaz Coefficient: 21.965
('Damping: ', 0.85000000000000031)
Estimated number of clusters: 28
Sil

ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [12]:
vectorised_test_documents = tfidf.transform(test_docs)

In [13]:
#List of document ids
documents = reuters.fileids()

In [14]:
train_docs_id = list(filter(lambda doc: doc.startswith("train"),
                            documents))
test_docs_id = list(filter(lambda doc: doc.startswith("test"),
                           documents))

In [15]:
# Transform multilabel labels
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform([reuters.categories(doc_id)
                                  for doc_id in train_docs_id])
test_labels = mlb.transform([reuters.categories(doc_id)
                             for doc_id in test_docs_id])